# Load Libraries and Functions

In [2]:
# System, Data, Time, and Spec Libraries
import os
import pandas as pd
import time
import random
from datetime import datetime, timedelta
import numpy as np 
from line_profiler import LineProfiler  # Code peformance
profiler = LineProfiler()
import warnings
warnings.filterwarnings('ignore')
import csv
#pd.set_option('display.max_colwidth', None)
import multiprocess as mp
num_cores = mp.cpu_count()

# Data Visualization Libraries
import matplotlib.pyplot as plt
from tabulate import tabulate
import plotly.express as px
import seaborn as sns
#from pandas.io.json import json_normalize  # Older version
from pandas import json_normalize  # Newer version
from pandas.plotting import parallel_coordinates


# Natural Language Processing Libraries
import json
import requests
import xml.etree.ElementTree as ET
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import words
import string
import nltk
from collections import OrderedDict
from nltk.tokenize import word_tokenize
from pandas import json_normalize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.util import ngrams
from nltk.corpus import stopwords
from string import punctuation
punctuation = set(punctuation)
punctuation.update({'_', '-','‘'})
english_words = set(words.words())
from fuzzywuzzy import process
#nltk.download('words')
#nltk.download('punkt')
#nltk.download('stopwords')

# SQL Interface Libraries
import pymysql as mysql
import mysql.connector
import pyodbc
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import event
from string import punctuation

# Machine Learning Libraries
import scipy.stats as stats
from scipy.stats import zscore
from scipy.stats import yeojohnson
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, PowerTransformer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay,roc_auc_score, roc_curve 
from sklearn.metrics import classification_report, mean_squared_error, f1_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from dmba import classificationSummary, AIC_score, BIC_score, plotDecisionTree,gainsChart
from scikitplot.metrics import plot_lift_curve, plot_cumulative_gain
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.metrics import specificity_score, sensitivity_score
from imblearn.under_sampling import RandomUnderSampler
from scikitplot.metrics import plot_lift_curve
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn import preprocessing
from sklearn import tree
from sklearn.model_selection import cross_val_score
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from dmba import stepwise_selection, classificationSummary, backward_elimination
from sklearn.decomposition import PCA
from sklearn.utils.random import sample_without_replacement
from sklearn.metrics import precision_score, recall_score

# Importing Custom Functions
import nbimporter
from Functions import nan_info

no display found. Using non-interactive Agg backend


# Test Join Outside of Database

In [68]:
# df1 = pd.read_csv('DataLibrary/events_table2023.csv')
# df2 = pd.read_csv('DataLibrary/patient_drugs_table2023.csv')
# df3 = pd.read_csv('DataLibrary/reactions_table2023.csv')

# # Merge df1 and df2
# merged_df1_df2 = pd.merge(df1, df2, on='event_id', how='inner')

# # Merge the result with df3
# final_merged_df = pd.merge(merged_df1_df2, df3, on='event_id', how='inner')

# # Display the final merged DataFrame
# print(final_merged_df)

In [6]:
merged_df1_df2.shape

(133423, 20)

### SQL Password

In [3]:
PASSWORD  = 'PASSWORD' 

## Investigate number of records in each database table

In [69]:
connection = mysql.connector.connect(
    host="localhost", user="root", password=PASSWORD, database="pharma_db"
)

cursor = connection.cursor()
sql_query = """SELECT COUNT(*) FROM patient_reactions"""


cursor.execute(sql_query)
result = cursor.fetchall()
column_names = [i[0] for i in cursor.description]


result_query_df = pd.DataFrame(result, columns=column_names)

cursor.close()
connection.close()

result_query_df

COUNT(*)
0    112551

# Get Dataset from SQL Database

In [70]:
# Connect to the MySQL server
connection = mysql.connector.connect(
    host="localhost", user="root", password=PASSWORD, database="pharma_db"
)

# Create a cursor object
cursor = connection.cursor()

In [15]:
# # Master Query from Data - goal

# master_query = """SELECT 
#                 a.serious_outcome,
#                 a.expedited,
#                 a.age,
#                 a.sex,
#                 a.year
#                 a.weight
#                 r.outcome,
#                 p.unit_price,
#                 p.generic_brand,
#                 l.ingredients,
#                 l.rxcui,
#                 l.set_id,
#                 d.manu_num,
#                 d.unii
#             FROM adverse_events a 
#                 INNER JOIN patients_reactions r ON a.event_id = r.event_id 
#                 INNER JOIN patients_drugs d ON r.event_id = d.event_id 
#                 INNER JOIN prices p ON d.ndc11 = p.ndc11
#                 INNER JOIN lables l ON p.ndc11 = l.ndc11
#             ORDER BY a.event_id DESC"""  # Still need to test and figure out

# Read in data from csv OR

In [ ]:
result_query_df = pd.read_csv('DataLibrary/result_query_df.csv')

# option 2- Read in data from SQL

In [76]:
start_time = time.time()

connection = mysql.connector.connect(
    host="localhost", user="root", password=PASSWORD, database="pharma_db"
)

cursor = connection.cursor()
sql_query = """SELECT 
d.med_product,
d.event_id,
d.manu_num,
a.serious_outcome,
a.expedited,
a.age,
a.sex,
a.year,
a.weight,
r.outcome
FROM adverse_events a 

INNER JOIN patient_reactions r ON a.event_id = r.event_id
INNER JOIN patient_drugs d ON a.event_id = d.event_id
ORDER BY a.event_id DESC
"""
cursor.execute(sql_query)
result = cursor.fetchall()
column_names = [i[0] for i in cursor.description]
result_query_df = pd.DataFrame(result, columns=column_names)

cursor.close()
connection.close()


end_time = time.time()
execution_time = end_time - start_time
print(execution_time)

result_query_df.head(10)


47.15399479866028


med_product  event_id  manu_num  serious_outcome  expedited  age  sex  \
0       ['stelara']     24685         1                2          1   64    1   
1       ['stelara']     24685         1                2          1   64    1   
2      ['remicade']     24685         1                2          1   64    1   
3  ['cyclosporine']     24685        16                2          1   64    1   
4    ['efalizumab']     24685         0                2          1   64    1   
5       ['stelara']     24685         1                2          1   64    1   
6       ['stelara']     24685         1                2          1   64    1   
7      ['remicade']     24685         1                2          1   64    1   
8  ['cyclosporine']     24685        16                2          1   64    1   
9    ['efalizumab']     24685         0                2          1   64    1   

   year  weight outcome  
0  2024      72   Fatal  
1  2024      72   Fatal  
2  2024      72   Fatal  
3  2024      72   Fatal  
4  2024      72   Fatal  
5  2024      72   Fatal  
6  2024      72   Fatal  
7  2024      72   Fatal  
8  2024      72   Fatal  
9  2024      72   Fatal

In [77]:
result_query_df.shape

(2653735, 10)

In [79]:
result_query_df = result_query_df.drop_duplicates()
result_query_df.shape

(99193, 10)

In [80]:
result_query_df.to_csv('DataLibrary/result_query_df.csv')

In [83]:
master_query_df = result_query_df

# Preparation for Data Split

In [84]:
master_query_df.head()

med_product  event_id  manu_num  serious_outcome  expedited  age  \
0        ['stelara']     24685         1                2          1   64   
2       ['remicade']     24685         1                2          1   64   
3   ['cyclosporine']     24685        16                2          1   64   
4     ['efalizumab']     24685         0                2          1   64   
10  ['lansoprazole']     24684        56                2          1   75   

    sex  year  weight outcome  
0     1  2024      72   Fatal  
2     1  2024      72   Fatal  
3     1  2024      72   Fatal  
4     1  2024      72   Fatal  
10    2  2024      63   Fatal

In [85]:
nan_info(master_query_df)

column_name  null_count  null_proportion
0      med_product           0              0.0
1         event_id           0              0.0
2         manu_num           0              0.0
3  serious_outcome           0              0.0
4        expedited           0              0.0
5              age           0              0.0
6              sex           0              0.0
7             year           0              0.0
8           weight           0              0.0
9          outcome           0              0.0

### Update Datatypes

In [86]:
master_query_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99193 entries, 0 to 2653734
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   med_product      99193 non-null  object
 1   event_id         99193 non-null  int64 
 2   manu_num         99193 non-null  int64 
 3   serious_outcome  99193 non-null  int64 
 4   expedited        99193 non-null  int64 
 5   age              99193 non-null  int64 
 6   sex              99193 non-null  int64 
 7   year             99193 non-null  int64 
 8   weight           99193 non-null  int64 
 9   outcome          99193 non-null  object
dtypes: int64(8), object(2)
memory usage: 8.3+ MB


### Define numerical, text, and categorical variables

In [88]:
# Can do without med_product for now to keep simple, will finish after module 5 submission
cats = ['med_product', 'sex', 'expedited']
nums = ['weight', 'age', 'manu_num'] 
#texts = ['']  # Do reaction and med product as text vector instead of binary matrix? only test out if time permits
#all_vars = cats+nums+texts"
all_vars = cats+nums

In [89]:
master_query_df.head()

med_product  event_id  manu_num  serious_outcome  expedited  age  \
0        ['stelara']     24685         1                2          1   64   
2       ['remicade']     24685         1                2          1   64   
3   ['cyclosporine']     24685        16                2          1   64   
4     ['efalizumab']     24685         0                2          1   64   
10  ['lansoprazole']     24684        56                2          1   75   

    sex  year  weight outcome  
0     1  2024      72   Fatal  
2     1  2024      72   Fatal  
3     1  2024      72   Fatal  
4     1  2024      72   Fatal  
10    2  2024      63   Fatal

In [90]:
master_query_df.columns

Index(['med_product', 'event_id', 'manu_num', 'serious_outcome', 'expedited',
       'age', 'sex', 'year', 'weight', 'outcome'],
      dtype='object')

In [91]:
all_vars

['med_product', 'sex', 'expedited', 'weight', 'age', 'manu_num']

### Create Transformation Pipeline

In [92]:
cats = ['med_product', 'sex', 'expedited']
nums = ['weight', 'age', 'manu_num'] 

# Create a categorical processing pipeline that uses one-hot encoding
# Dropping binary columns and drop first of each level** NEED TO ADD**
cat_pipe = Pipeline([('encoder', OneHotEncoder(drop='if_binary'))])

# Create a numerical processing pipeline that uses skewness correction/center/scale.
num_pipe = Pipeline([('skew_standardize', PowerTransformer())])

# Create a text token processing step to vectorize tokens
#text_pipe = Pipeline([('vector', tf_idf function())])

# Combine pipeline steps
all_pipe = make_pipeline(
    ColumnTransformer([
        ('cat', cat_pipe, cats),  # Apply categorical pipeline to categorical columns
        ('num', num_pipe, nums),  # Apply numerical pipeline to numerical columns
     #   ('text', text_pipe, texts)  # Apply text pipeline to text columns
    ],
    verbose_feature_names_out=False)  # Set this to False to avoid verbose feature names
)
# Verify steps

all_pipe.named_steps

{'columntransformer': ColumnTransformer(transformers=[('cat',
                                  Pipeline(steps=[('encoder',
                                                   OneHotEncoder(drop='if_binary'))]),
                                  ['med_product', 'sex', 'expedited']),
                                 ('num',
                                  Pipeline(steps=[('skew_standardize',
                                                   PowerTransformer())]),
                                  ['weight', 'age', 'manu_num'])],
                   verbose_feature_names_out=False)}

# Split Data into Training/Validation/Testing

In [113]:
# Split the data into features (X) and the target variable (y).
X = master_query_df[all_vars]

#Define outcome variable
y = master_query_df[['serious_outcome']]  # Need to Decide 5 Level or 3 Level

#Split data
X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=0.8, random_state = 2)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state = 2)

In [94]:
X_val.head()

med_product  sex  expedited  weight  age  manu_num
1617080       ['rituximab']    1          1      78   71         1
2437331    ['methotrexate']    2          1      68   43        21
432915   ['metoclopramide']    2          1      89   69         9
2521465          ['viread']    1          1      60   48         1
1255957         ['lovenox']    2          1      63   70         1

In [95]:
X_train.shape, y_train.shape

((79354, 6), (79354, 1))

In [96]:
X_test.shape, y_test.shape

((9920, 6), (9920, 1))

In [97]:
X_val.shape, y_val.shape

((9919, 6), (9919, 1))

## Apply Pipeline to Splits Separately to Prevent Data Leakage

In [98]:
X_train.head()

med_product  sex  expedited  weight  age  manu_num
684183   ['carboplatin']    2          1      71   52        11
34813     ['gadobutrol']    2          1      59   16         3
440773       ['januvia']    1          1      42   77         1
2619947       ['prolia']    1          1      81   75         1
406429       ['ocrevus']    1          1      95   71         1

In [99]:
X_train.columns

Index(['med_product', 'sex', 'expedited', 'weight', 'age', 'manu_num'], dtype='object')

In [105]:

#Apply pipeline to all of X
X_train_fit = all_pipe.fit_transform(X_train)
#Get feature names out from fit and create as new list
X_train_cols = X_train_fit.get_feature_names_out().tolist()
#X_train_pipe = pd.DataFrame(all_pipe.fit_transform(X_train), columns = X_train_cols)

AttributeError: get_feature_names_out not found

In [59]:
#Apply pipeline to all of X
X_test_fit = all_pipe.fit(X_test)
#Get feature names out from fit and create as new list
X_test_cols = X_test_fit.get_feature_names_out().tolist()
X_test_pipe = pd.DataFrame(all_pipe.fit_transform(X_test), columns = X_cols)

ValueError: A given column is not a column of the dataframe

In [60]:
#Apply pipeline to all of X
X_val_fit = all_pipe.fit(X_val)
#Get feature names out from fit and create as new list
X_val_cols = X_val_fit.get_feature_names_out().tolist()
X_val_pipe = pd.DataFrame(all_pipe.fit_transform(X_val), columns = X_cols)

ValueError: A given column is not a column of the dataframe

## Undersample Training Data to Balance Outcome

In [154]:
# Create a RandomUnderSampler instance with a specified random seed and sampling strategy
rus = RandomUnderSampler(random_state = 1, sampling_strategy='majority')

# Perform random under-sampling on the training dataset
X_train_under, y_train_under = rus.fit_resample(X_train_pipe, y_train)

# Multiclass Classification Model Training

## White Box Models

### LOGISTIC REGRESSION

### Logistic Regression L1 Regularization

In [155]:
X_train.dtypes

med_product_filgrastim    float64
med_product_iressa        float64
med_product_lenvatinib    float64
med_product_xolair        float64
med_product_xyrem         float64
sex_2                     float64
expedited_2               float64
weight                    float64
year                      float64
dtype: object

In [114]:
log_l1 = LogisticRegression(solver='saga', penalty='l1', random_state=1)
log_l1.fit(X_train, y_train)

# Intercept Log-Odds and Odds
print(log_l1.intercept_ , np.exp(log_l1.intercept_))


ValueError: could not convert string to float: "['carboplatin']"

### Logistic Regression L2 Regularization

In [105]:
log_l2 = LogisticRegression(solver='saga', penalty='l2', random_state=1)
log_l2.fit(X_train_under, y_train_under.values.ravel())

# Intercept Log-Odds and Odds
print(log_l2.intercept_ , np.exp(log_l2.intercept_))

[-0.12139508] [0.88568397]


### Logistic Regression L1 w/ 10-k CV

In [106]:
# Create Logistic Regression model with L1 regularization
log_l1_cv = LogisticRegressionCV(solver = 'saga', penalty = 'l1', cv = 2, random_state = 1)
# Fit the model to the training data
log_l1_cv.fit(X_train_under, y_train_under.values.ravel())

# Intercept Log-Odds and Odds
print(log_l1_cv.intercept_ , np.exp(log_l1_cv.intercept_))

[0.03186184] [1.03237486]


In [107]:
# Create table of coefficient odds
d = {'Feature': pd.Series(X_train_under.columns), 'LogOdds': pd.Series(log_l1_cv.coef_[0])}
df = pd.DataFrame(data=d).reindex(d['LogOdds'].abs().sort_values(ascending=False).index)
df['Odds'] = np.exp(df['LogOdds'])
df

Feature   LogOdds      Odds
7                  weight -1.492369  0.224839
0  med_product_filgrastim  0.000000  1.000000
1      med_product_iressa  0.000000  1.000000
2  med_product_lenvatinib  0.000000  1.000000
3      med_product_xolair  0.000000  1.000000
4       med_product_xyrem  0.000000  1.000000
5                   sex_2  0.000000  1.000000
6             expedited_2  0.000000  1.000000
8                    year  0.000000  1.000000

In [100]:
# Cross val performance

### Logistic Regression Elastic Net

In [101]:
param_grid = {
    'l1_ratio': [i / 9 for i in range(10)],  # 10 values from 0 to 1 (0, 0.1, 0.2, ..., 1.0)
    'C': [0.01, 0.1, 1, 10, 100]  # Different strengths of regularization
}

log_reg_elasticnet = LogisticRegression(
    penalty='elasticnet',  # Use Elastic Net regularization
    solver='saga',        # Solver that supports Elastic Net
    random_state=1
)

grid_search = GridSearchCV(
    estimator=log_reg_elasticnet,
    param_grid=param_grid,
    scoring='accuracy',   # Or another metric of choice
    cv=5,                 # Number of cross-validation folds
    n_jobs=-1              # Use all available CPUs
)

grid_search.fit(X_train_under, y_train_under.values.ravel())


best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

best_score = grid_search.best_score_
print(f"Best cross-validation score: {best_score:.4f}")

best_model = grid_search.best_estimator_

Best parameters: {'C': 0.1, 'l1_ratio': 0.0}
Best cross-validation score: 1.0000


### SINGLE DECISION TREE

In [ ]:
# Define a grid of hyperparameters to search for in tree
param_grid = {
    'max_depth' : [1,2,3,4,5],
    'min_samples_leaf' : [1,2,3,4,5]
    
}
# Create a GridSearchCV object using and the defined parameter grid
tree1_search = GridSearchCV(DecisionTreeClassifier(random_state=1), param_grid, cv=10, n_jobs=-1)
# Fit the GridSearchCV to the balanced training data to find the best hyperparameters
tree1_search.fit(X_train_under, y_train_under.values.ravel())
# Get the best hyperparameters found by the GridSearch
tree1_search.best_params_

In [ ]:
tree1 = DecisionTreeClassifier(max_depth = 4, min_samples_leaf = 4, random_state = 1).fit(X_train_under, y_train_under)

In [ ]:
# Get feature importance scores
importances=tree1.feature_importances_
feature_importance_pairs=list(zip(X_train_under.columns, importances))

# Sort in descending order
sorted_feature_importance_pairs = sorted(feature_importance_pairs, key=lambda x: x[1], reverse=True)

# Print scores
print("Feature Importance Scores")
for feature, importance in sorted_feature_importance_pairs:
    print(f"{feature}: {importance:.4f}")

In [ ]:
# Cross val performance

## Ensembles

### Random Forest Classifier w/ 10-k CV

In [ ]:
#Random Forest
# Create a Random Forest classifier with 100 trees
random_forest = RandomForestClassifier(n_estimators=1000, random_state=1)  

# Fit (train) the Random Forest classifier on the balanced training data
random_forest.fit(X_train_under, y_train_under.values.ravel())

In [ ]:
# Get feature importance scores
importances=random_forest.feature_importances_
feature_importance_pairs=list(zip(X_train_under.columns, importances))

# Sort in descending order
sorted_feature_importance_pairs = sorted(feature_importance_pairs, key=lambda x: x[1], reverse=True)

# Print scores
print("Feature Importance Scores")
for feature, importance in sorted_feature_importance_pairs:
    print(f"{feature}: {importance:.4f}")

In [ ]:
# Extract feature names and importances
feature_names, importances = zip(*sorted_feature_importance_pairs)

# Create a horizontal bar plot
plt.figure(figsize=(14, 12))
#plt.barh(range(len(feature_names)), importances, align='center')
#plt.yticks(range(len(feature_names)), feature_names)

plt.barh(range(len(feature_names)), importances, align='center')
plt.yticks(range(len(feature_names)), feature_names)

plt.xlabel('Feature Importance')
plt.gca().invert_yaxis()  # Invert the y-axis to show the most important features at the top
#plt.show()
plt.savefig('featimp.png')

In [ ]:
# Cross val peformance

## Gradient Boosted Tree w/ 10-k CV

In [ ]:
# Define a grid of hyperparameters to search for in the Gradient Boosting Classifier
param_grid = {
    'max_depth' : [1,2,3,4,5],
    'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1],
    'n_estimators' : [20,21,22,23,24],
}
# Create a GridSearchCV object using GradientBoostingClassifier and the defined parameter grid
tree_search = GridSearchCV(GradientBoostingClassifier(random_state=1), param_grid, cv=10, n_jobs=-1)
# Fit the GridSearchCV to the balanced training data to find the best hyperparameters
tree_search.fit(X_train_under, y_train_under.values.ravel())
# Get the best hyperparameters found by the GridSearch
tree_search.best_params_

In [ ]:
# Cross val performance

## Non-parametric

### K-Nearest Neighbors Classifier

In [ ]:
# Initialize an empty list to store KNN results
knn_results = []
for k in range (1, 50):
    # Create a K-Nearest Neighbors model with k neighbors and fit it to the balanced training data
    knn_mod = KNeighborsClassifier(n_neighbors = k).fit(X_train_under, y_train_under.values.ravel())
    # Calculate and append results to the list
    knn_results.append({
        'k': k,
        'Sens': sensitivity_score(y_test_imp.values.ravel(), knn_mod.predict(X_test_combo)),
        'Acc': accuracy_score(y_test_imp.values.ravel(), knn_mod.predict(X_test_combo)) 
    })
# Create a DataFrame from the list of KNN results    
knn_results = pd.DataFrame(knn_results)
warnings.filterwarnings('ignore')

In [ ]:
knn_results

In [ ]:
# Create a K-Nearest Neighbors (KNN) classifier with 6 neighbors and fit it to the balanced training data
knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train_under, y_train_under.values.ravel())

## Neural Network

In [ ]:
# train

In [ ]:
# feature importance

In [ ]:
# 

# Validation Data Tuning

# Testing Performance

## Logistic Regression

### Predictions

In [149]:
X_train_pipe

med_product_filgrastim  med_product_iressa  med_product_lenvatinib  \
0                      0.0                 1.0                     0.0   
1                      0.0                 1.0                     0.0   
2                      1.0                 0.0                     0.0   
3                      0.0                 1.0                     0.0   
4                      0.0                 0.0                     1.0   
5                      0.0                 0.0                     0.0   
6                      0.0                 1.0                     0.0   
7                      0.0                 0.0                     0.0   
8                      0.0                 0.0                     0.0   
9                      0.0                 0.0                     1.0   
10                     0.0                 0.0                     0.0   
11                     0.0                 1.0                     0.0   
12                     0.0                 0.0                     0.0   
13                     0.0                 0.0                     0.0   
14                     0.0                 0.0                     0.0   
15                     0.0                 1.0                     0.0   

    med_product_xolair  med_product_xyrem  sex_2  expedited_2    weight  year  
0                  0.0                0.0    1.0          0.0 -1.159326   0.0  
1                  0.0                0.0    1.0          0.0 -1.159326   0.0  
2                  0.0                0.0    1.0          0.0  0.563567   0.0  
3                  0.0                0.0    1.0          0.0 -1.159326   0.0  
4                  0.0                0.0    0.0          1.0 -0.383712   0.0  
5                  1.0                0.0    0.0          0.0  1.019654   0.0  
6                  0.0                0.0    1.0          0.0 -1.159326   0.0  
7                  1.0                0.0    0.0          0.0  1.019654   0.0  
8                  1.0                0.0    0.0          0.0  1.019654   0.0  
9                  0.0                0.0    0.0          1.0 -0.383712   0.0  
10                 0.0                1.0    1.0          0.0  1.041891   0.0  
11                 0.0                0.0    1.0          0.0 -1.159326   0.0  
12                 1.0                0.0    0.0          0.0  1.019654   0.0  
13                 1.0                0.0    0.0          0.0  1.019654   0.0  
14                 1.0                0.0    0.0          0.0  1.019654   0.0  
15                 0.0                0.0    1.0          0.0 -1.159326   0.0

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- sex_1
Feature names seen at fit time, yet now missing:
- med_product_filgrastim
- med_product_iressa
- med_product_lenvatinib
- med_product_xyrem
- sex_2


### Confusion Matrix

In [ ]:
#Logistic Regression
log_cm = confusion_matrix(y_true = y_test, y_pred = log_l1.predict(X_test_log), labels = log_l2_bal.classes_)
log_disp = ConfusionMatrixDisplay(confusion_matrix= log_cm_bal, display_labels=log_l2_bal.classes_)
log_disp.plot()
#plt.show()
plt.savefig('lr_cm.png')

In [ ]:
# classification summary

In [ ]:
# curve

In [ ]:
#Sensitivity, specificity, and accuracy for log reg
logl1_sens = sensitivity_score(y_test, log_l1.predict(X_test))
logl1_spec = specificity_score(y_test, log_l1.predict(X_test))
logl1_acc = accuracy_score(y_test, log_l1.predict(X_test))
logl1_prec = precision_score(y_test, log_l1.predict(X_test)) 
logl1_rec = recall_score(y_test, log_l1.predict(X_test))
logl1_f1 = f1_score(y_test, log_l1.predict(X_test))
logl1_sens, logl1_spec, logl1_acc, logl1_prec, logl1_rec, logl1_f1

## Decision Tree

### Predictions

### Confusion Matrix

In [ ]:
#Confusion matrix for tree
tree_cm = confusion_matrix(y_true = y_test, y_pred = tree1.predict(X_test), labels = tree1.classes_)
tree_disp = ConfusionMatrixDisplay(confusion_matrix= tree_cm, display_labels=tree1.classes_)
tree_disp.plot()
plt.show() 

In [ ]:
# classification summary

In [ ]:
# curve

In [ ]:
#Sensitivity, specificity, and accuracy for decision tree
tree1_sens = sensitivity_score(y_test, tree1.predict(X_test))
tree1_spec = specificity_score(y_test, tree1.predict(X_test))
tree1_acc = accuracy_score(y_test, tree1.predict(X_test))
tree1_prec = precision_score(y_test, tree1.predict(X_test)) 
tree1_rec = recall_score(y_test, tree1.predict(X_test))
tree1_f1 = f1_score(y_test, tree1.predict(X_test))
tree1_sens, tree1_spec, tree1_acc, tree1_prec, tree1_rec, tree1_f1

## Random Forest

### Predictions

### Confusion Matrix

In [ ]:
# Compute the confusion matrix for RF 
rf_cm = confusion_matrix(y_true = y_test, y_pred = random_forest.predict(X_test), labels = random_forest.classes_)
# Create a ConfusionMatrixDisplay object for visualization
rf_disp = ConfusionMatrixDisplay(confusion_matrix= rf_cm, display_labels=random_forest.classes_)
rf_disp.plot()
warnings.filterwarnings('ignore')
#plt.show()
#plt.savefig('rf_cm.png')

In [1]:
# classification summary

In [ ]:
# curve

In [ ]:
#Sensitivity, specificity, and accuracy for RF
rf_sens = sensitivity_score(y_test, random_forest.predict(X_test))
rf_spec = specificity_score(y_test, random_forest.predict(X_test))
rf_acc = accuracy_score(y_test, random_forest.predict(X_test))
rf_prec = precision_score(y_test, random_forest.predict(X_test)) 
rf_rec = recall_score(y_test, random_forest.predict(X_test))
rf_f1 = f1_score(y_test, random_forest.predict(X_test))
warnings.filterwarnings('ignore')
rf_sens, rf_spec, rf_acc, rf_prec, rf_rec, rf_f1

## Gradient Boosted Tree

### Predictions

### Confusion Matrix

In [ ]:
#Confusion matrix for tree
tree_cm = confusion_matrix(y_true = y_test, y_pred = tree_search.predict(X_test), labels = tree_search.classes_)
tree_disp = ConfusionMatrixDisplay(confusion_matrix= tree_cm, display_labels=tree_search.classes_)
tree_disp.plot()
plt.show() 

In [2]:
# classification summary

In [ ]:
# curve

In [ ]:
#Sensitivity, specificity, and accuracy for decision tree 
tree_sens = sensitivity_score(y_test, tree_search.predict(X_test))
tree_spec = specificity_score(y_test, tree_search.predict(X_test))
tree_acc = accuracy_score(y_test, tree_search.predict(X_test))
tree_prec = precision_score(y_test, tree_search.predict(X_test)) 
tree_rec = recall_score(y_test, tree_search.predict(X_test))
tree_f1 = f1_score(y_test, tree_search.predict(X_test))
tree_sens, tree_spec, tree_acc, tree_prec, tree_rec, tree_f1

## K-Nearest Neighbors

### Predictions

### Confusion Matrix

In [ ]:
#KNN
# Compute the confusion matrix for KNN 
knn_cm = confusion_matrix(y_true = y_test, y_pred = knn.predict(X_test), labels = knn.classes_)
# Create a ConfusionMatrixDisplay object for visualization
knn_disp = ConfusionMatrixDisplay(confusion_matrix= knn_cm, display_labels=knn.classes_)
knn_disp.plot()
warnings.filterwarnings('ignore')
plt.show()

In [ ]:
# Classification summary

In [ ]:
# Curve

In [ ]:
#Sensitivity, specificity, and accuracy for KNN 
knn_sens = sensitivity_score(y_test, knn.predict(X_test))
knn_spec = specificity_score(y_test, knn.predict(X_test))
knn_acc = accuracy_score(y_test, knn.predict(X_test))
knn_prec = precision_score(y_test, knn.predict(X_test)) 
knn_rec = recall_score(y_test, knn.predict(X_test))
knn_f1 = f1_score(y_test, knn.predict(X_test))
warnings.filterwarnings('ignore')
knn_sens, knn_spec, knn_acc, knn_prec, knn_rec, knn_f1

## Neural Network

### Predictions

### Confusion Matrix

In [ ]:
# Classification summary

In [ ]:
# Curve

# Performance Metrics Table

In [ ]:
# Create a table to display performance metrics for different models on the validation dataset

val_performance = [
{'Model': 'Neural Net', 'Test Sensitivity': nn_sens, 'Test Specificity':nn_spec, 
 'Accuracy': nn_acc, 'Test Precision': nn_prec, 'Test Recall': nn_rec, 'F1 Score': nn_f1},
    {'Model': 'Logistic Regression', 'Test Sensitivity': logl1_sens, 'Test Specificity':logl1_spec, 
 'Accuracy': logl1_acc, 'Test Precision': logl1_prec, 'Test Recall': logl1_rec,  'F1 Score': logl1_f1},
    {'Model': 'Boosted Tree', 'Test Sensitivity': tree_sens, 'Test Specificity':tree_spec, 
 'Accuracy': tree_acc, 'Test Precision': tree_prec, 'Test Recall': tree_rec,  'F1 Score': tree_f1},
    {'Model': 'Single Tree', 'Test Sensitivity': tree1_sens, 'Test Specificity':tree1_spec, 
 'Accuracy': tree1_acc, 'Test Precision': tree1_prec, 'Test Recall': tree1_rec, 'F1 Score': tree1_f1},
    {'Model': 'Random Forest', 'Test Sensitivity': rf_sens, 'Test Specificity':rf_spec, 
 'Accuracy': rf_acc, 'Test Precision': rf_prec, 'Test Recall': rf_rec, 'F1 Score': rf_f1},
    {'Model': 'K-Nearest Neighbors', 'Test Sensitivity': knn_sens, 'Test Specificity':knn_spec, 
 'Accuracy': knn_acc, 'Test Precision': knn_prec, 'Test Recall': knn_rec,  'F1 Score': knn_f1},
    
]
# Create a formatted table using tabulate and specify the format as 'fancy_grid'
table = tabulate(val_performance, headers='keys', tablefmt='fancy_grid')
# Display the comparison table
print(table)